In [ ]:
pip install avalanche-lib==0.5

In [ ]:
from torch.utils.data import Subset
from avalanche.benchmarks.classic.ccifar100 import SplitCIFAR100
import numpy as np

def create_imbalanced_subset(dataset, class_counts):
    """Create an imbalanced subset of the dataset given class counts."""
    indices_per_class = {label: [] for label in range(100)}


    for idx in range(len(dataset)):
        label = dataset.targets[idx] if hasattr(dataset, 'targets') else dataset[idx][1]
        indices_per_class[label].append(idx)


    selected_indices = []
    for class_label, count in class_counts.items():
        available_count = len(indices_per_class[class_label])
        selected_count = min(count, available_count)
        selected_indices.extend(np.random.choice(indices_per_class[class_label], selected_count, replace=False))

    return np.array(selected_indices)


num_phases = 10
initial_count = 500 
decay_factor = 0.7


phase_class_counts = []
for i in range(num_phases):
    count = int(initial_count * (decay_factor ** i)) 
    class_counts = {label: count for label in range(100)}
    phase_class_counts.append(class_counts)


scenario = SplitCIFAR100(n_experiences=num_phases)


imbalanced_scenarios = []
for exp_id in range(num_phases):
    train_dataset = scenario.train_stream[exp_id].dataset
    counts = phase_class_counts[exp_id]


    selected_indices = create_imbalanced_subset(train_dataset, counts)
    imbalanced_train_subset = Subset(train_dataset, selected_indices)


    imbalanced_scenarios.append(imbalanced_train_subset)


    print(f"Phase {exp_id + 1}: Number of samples = {len(imbalanced_train_subset)}")


In [ ]:
from torch.utils.data import Subset
from avalanche.benchmarks.classic.ccifar100 import SplitCIFAR100
import numpy as np

def create_imbalanced_subset(dataset, class_counts):
    indices_per_class = {label: [] for label in range(100)}
    
    for idx in range(len(dataset)):
        label = dataset.targets[idx] if hasattr(dataset, 'targets') else dataset[idx][1]
        indices_per_class[label].append(idx)

    selected_indices = []
    for class_label, count in class_counts.items():
        available_count = len(indices_per_class[class_label])
        selected_count = min(count, available_count)
        selected_indices.extend(np.random.choice(indices_per_class[class_label], selected_count, replace=False))

    return np.array(selected_indices)

num_phases = 10
classes_per_phase = 100 // num_phases
initial_count = 500
decay_factor = 0.7

phase_class_counts = []
phase_classes = []

for i in range(num_phases):
    start_class = i * classes_per_phase
    end_class = start_class + classes_per_phase
    current_classes = list(range(start_class, end_class))
    phase_classes.append(current_classes)
    
    count = int(initial_count * (decay_factor ** i))
    class_counts = {label: count for label in current_classes}
    phase_class_counts.append(class_counts)

scenario = SplitCIFAR100(n_experiences=num_phases, fixed_class_order=list(range(100)))

imbalanced_scenarios = []
for exp_id in range(num_phases):
    train_dataset = scenario.train_stream[exp_id].dataset
    counts = phase_class_counts[exp_id]

    selected_indices = create_imbalanced_subset(train_dataset, counts)
    imbalanced_train_subset = Subset(train_dataset, selected_indices)

    imbalanced_scenarios.append(imbalanced_train_subset)

    print(f"Phase {exp_id + 1}: Classes = {phase_classes[exp_id]}, Number of samples = {len(imbalanced_train_subset)}")


In [ ]:
pip install pytorchcv==0.0.67

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torchvision import transforms
from torch.utils.data import Subset
from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.training.supervised import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics
from avalanche.logging import InteractiveLogger

# Step 1: Define a feature extractor using pretrained ResNet18 with fixed weights
class FixedFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(FixedFeatureExtractor, self).__init__()
        resnet = resnet18(pretrained=pretrained)
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # Remove the final classification layer
        for param in self.features.parameters():
            param.requires_grad = False  # Freeze the feature extractor

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # Flatten the output to [batch_size, 512]
        return x

# Step 2: Define a model that combines the fixed feature extractor with a trainable classification head
class FeatureExtractorWithHead(nn.Module):
    def __init__(self, num_classes=100):
        super(FeatureExtractorWithHead, self).__init__()
        self.feature_extractor = FixedFeatureExtractor()
        self.classifier = nn.Linear(512, num_classes)  # ResNet18 outputs 512-dimensional features

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

# Step 3: Prepare the CIFAR-100 benchmark split into 10 phases with resizing transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit ResNet18's expected input size
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # CIFAR-100 normalization values
])

benchmark = SplitCIFAR100(n_experiences=10, fixed_class_order=list(range(100)), train_transform=transform, eval_transform=transform)

# Step 4: Initialize the model, optimizer, criterion, and EWC strategy
model = FeatureExtractorWithHead(num_classes=benchmark.n_classes)
optimizer = optim.SGD(model.classifier.parameters(), lr=0.01, momentum=0.9)  # Only train the classifier
criterion = nn.CrossEntropyLoss()

# Step 5: Set up evaluation metrics and logging
interactive_logger = InteractiveLogger()
evaluation_plugin = EvaluationPlugin(
    accuracy_metrics(epoch=True, stream=True),
    loss_metrics(epoch=True, stream=True),
    loggers=[interactive_logger]
)

# Step 6: Initialize EWC strategy for continual learning
ewc_lambda = 0.001  # Regularization strength for EWC
strategy = EWC(
    model, optimizer, criterion,
    ewc_lambda=ewc_lambda,
    train_epochs=5,  # Number of training epochs for each phase
    eval_every=1,  # Evaluate after each epoch
    plugins=[evaluation_plugin]
)

# Step 7: Training loop over the split CIFAR-100 benchmark
for experience in benchmark.train_stream:
    print(f"Training on phase {experience.current_experience + 1}")
    strategy.train(experience)
    strategy.eval(benchmark.test_stream)


Files already downloaded and verified
Files already downloaded and verified
Training on phase 1
-- >> Start of training phase << --
-- >> Start of training phase << --
-- >> Start of eval phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from train stream --
0it [00:00, ?it/s]-- Starting eval on experience 0 (Task 0) from train stream --
100%|██████████| 5000/5000 [01:06<00:00, 75.72it/s]
> Eval on experience 0 (Task 0) from train stream ended.
100%|██████████| 5000/5000 [01:06<00:00, 75.72it/s]
> Eval on experience 0 (Task 0) from train stream ended.
	Loss_Exp/eval_phase/train_stream/Task000/Exp000 = 4.8253
	Top1_Acc_Exp/eval_phase/train_stream/Task000/Exp000 = 0.0032
-- >> End of eval phase << --
	Loss_Stream/eval_phase/train_stream/Task000 = 4.8253
	Top1_Acc_Stream/eval_phase/train_stream/Task000 = 0.0032
-- >> End of eval phase << --
	Loss_Stream/eval_phase/train_stream/Task000 = 4.8253
	Top1_Acc_Stream/eval_phase/train_stream/Task000 = 0.0032
10

KeyboardInterrupt: 

 56%|█████▋    | 2825/5000 [00:56<00:32, 67.48it/s]